In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
test_data.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
import re

train_data['Title'] = train_data['Name'].apply(lambda name: re.search(' ([A-Za-z]+)\.', name).group(1))
train_data['HasCabin'] = train_data['Cabin'].apply(lambda x: 0 if x is np.nan else 1)

train_data['Age'] = train_data.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

In [7]:
X_train = train_data.drop(['PassengerId', 'Survived', 'Name', 'Cabin', 'Ticket'], axis=1)
y_train = train_data['Survived']

In [8]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [9]:
test_data['Title'] = test_data['Name'].apply(lambda name: re.search(' ([A-Za-z]+)\.', name).group(1))
test_data['HasCabin'] = test_data['Cabin'].apply(lambda x: 0 if x is np.nan else 1)

test_data['Age'] = test_data.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median() if len(x.dropna()) > 0 else test_data['Age'].median()))
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

In [10]:
X_test = test_data.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)

In [11]:
X_train.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,HasCabin
0,3,male,22.0,1,0,7.2500,S,Mr,0
1,1,female,38.0,1,0,71.2833,C,Mrs,1
2,3,female,26.0,0,0,7.9250,S,Miss,0
3,1,female,35.0,1,0,53.1000,S,Mrs,1
4,3,male,35.0,0,0,8.0500,S,Mr,0


In [12]:
X_test.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,HasCabin
0,3,male,34.5,0,0,7.8292,Q,Mr,0
1,3,female,47.0,1,0,7.0000,S,Mrs,0
2,2,male,62.0,0,0,9.6875,Q,Mr,0
3,3,male,27.0,0,0,8.6625,S,Mr,0
4,3,female,22.0,1,1,12.2875,S,Mrs,0


In [13]:
title_categories = set(list(X_train['Title'].unique()) + list(X_test['Title'].unique()))
X_train['Title'] = pd.Categorical(X_train['Title'], categories=title_categories)
X_test['Title'] = pd.Categorical(X_test['Title'], categories=title_categories)

In [14]:
features = X_train.select_dtypes(include=[object, 'category']).columns
X_train_dummies = pd.get_dummies(X_train, columns=features)
X_test_dummies = pd.get_dummies(X_test, columns=features)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = { 
    'n_estimators': [10, 25, 50, 100, 500],
    'max_depth' : [4, 6, 8, 10],
    'criterion' :['gini', 'entropy']
}

rfc = RandomForestClassifier(random_state=123)

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, n_jobs=-1)
CV_rfc.fit(X_train_dummies, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=123),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [10, 25, 50, 100, 500]})

In [16]:
CV_rfc.best_params_

{'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 25}

In [17]:
rfc = CV_rfc.best_estimator_
predictions = rfc.predict(X_test_dummies)
CV_rfc.best_score_

0.8338710689849979

In [18]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)